In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [71]:
df = pd.read_csv("final.csv")

C:\Users\Shefi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [72]:
df.head()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,SimillarHTTP,Inbound,Label
0,12165,172.16.0.5-192.168.50.4-836-49970-17,172.16.0.5,836,192.168.50.4,49970,17,2018-11-03 10:29:20.733504,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,LDAP
1,11565,172.16.0.5-192.168.50.4-540-56876-17,172.16.0.5,540,192.168.50.4,56876,17,2018-11-03 10:26:06.491636,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,LDAP
2,22204,172.16.0.5-192.168.50.4-666-53544-17,172.16.0.5,666,192.168.50.4,53544,17,2018-11-03 10:09:11.392130,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,NetBIOS
3,4613,172.16.0.5-192.168.50.4-515-38921-17,172.16.0.5,515,192.168.50.4,38921,17,2018-11-03 10:09:22.262430,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,NetBIOS
4,17607,172.16.0.5-192.168.50.4-913-20813-17,172.16.0.5,913,192.168.50.4,20813,17,2018-11-03 10:23:53.383562,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1,LDAP


In [73]:
df.columns

Index(['Unnamed: 0', 'Flow ID', ' Source IP', ' Source Port',
       ' Destination IP', ' Destination Port', ' Protocol', ' Timestamp',
       ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std',
       'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s',
       ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std',
       ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean',
       ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags',
       ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags',
       ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s',
       ' Bwd Packets/s', ' Min Packet Len

## Pre processing

In [74]:
import numpy as np
print(f"Missing values: {df.isnull().sum().sum()}")
df.replace([np.inf, -np.inf], np.nan, inplace=True)
print(f"Missing values: {df.isnull().sum().sum()}")

Missing values: 0
Missing values: 0


In [75]:
print(df[' Bwd PSH Flags'].value_counts())
print(df[' Bwd URG Flags'].value_counts())
print(df['Fwd Avg Bytes/Bulk'].value_counts())
print(df[' Fwd Avg Packets/Bulk'].value_counts())
print(df[' Fwd Avg Bulk Rate'].value_counts())
print(df[' Bwd Avg Bytes/Bulk'].value_counts())
print(df[' Bwd Avg Packets/Bulk'].value_counts())
print(df['Bwd Avg Bulk Rate'].value_counts())
    

0    167402
Name:  Bwd PSH Flags, dtype: int64
0    167402
Name:  Bwd URG Flags, dtype: int64
0    167402
Name: Fwd Avg Bytes/Bulk, dtype: int64
0    167402
Name:  Fwd Avg Packets/Bulk, dtype: int64
0    167402
Name:  Fwd Avg Bulk Rate, dtype: int64
0    167402
Name:  Bwd Avg Bytes/Bulk, dtype: int64
0    167402
Name:  Bwd Avg Packets/Bulk, dtype: int64
0    167402
Name: Bwd Avg Bulk Rate, dtype: int64


In [76]:
df.drop([' Bwd PSH Flags'], axis=1, inplace=True)
df.drop([' Bwd URG Flags'], axis=1, inplace=True)
df.drop(['Fwd Avg Bytes/Bulk'], axis=1, inplace=True)
df.drop([' Fwd Avg Packets/Bulk'], axis=1, inplace=True)
df.drop([' Fwd Avg Bulk Rate'], axis=1, inplace=True)
df.drop([' Bwd Avg Bytes/Bulk'], axis=1, inplace=True)
df.drop([' Bwd Avg Packets/Bulk'], axis=1, inplace=True)
df.drop(['Bwd Avg Bulk Rate'], axis=1, inplace=True)
df.drop([" Source IP", " Destination IP", " Timestamp", "Flow ID"], inplace=True, axis=1)
df.drop(['SimillarHTTP'], inplace=True, axis=1)

## Feature Selection using genetic algorithm

In [47]:
from feature_selection_ga import FeatureSelectionGA, FitnessFunction

class CustomFitnessFunctionClass:
    def __init__(self,n_total_features,n_splits = 3, alpha=0.01, *args,**kwargs):
        self.n_splits = n_splits
        self.alpha = alpha
        self.n_total_features = n_total_features

    def calculate_fitness(self,model,x,y):
        alpha = self.alpha
        total_features = self.n_total_features

        cv_set = np.repeat(-1.,x.shape[0])
        skf = StratifiedKFold(n_splits = self.n_splits)
        for train_index,test_index in skf.split(x,y):
            x_train,x_test = x[train_index],x[test_index]
            y_train,y_test = y[train_index],y[test_index]
            if x_train.shape[0] != y_train.shape[0]:
                raise Exception()
            model.fit(x_train,y_train)
            predicted_y = model.predict(x_test)
            cv_set[test_index] = predicted_y
        
        P = accuracy_score(y, cv_set)
        fitness = (alpha*(1.0 - P) + (1.0 - alpha)*(1.0 - (x.shape[1])/total_features))
        return fitness


In [48]:
df1=df
df1[" Label"] = df1[" Label"].astype('category')
df1[" Label"] = df1[" Label"].astype('category')
df1["Label code"] = df1[" Label"].cat.codes

In [49]:
XX = df1.iloc[0:100,0:73].to_numpy()
yy = df1["Label code"][0:100]

model = RandomForestClassifier()
fsga = FeatureSelectionGA(model,XX,yy, ff_obj = FitnessFunction())
pop = fsga.generate(100)

C:\Users\Shefi\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FeatureSelect' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\Shefi\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\Shefi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\Shefi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\U

EVOLVING.......


C:\Users\Shefi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\Shefi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\Shefi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\Shefi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The mi

C:\Users\Shefi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\Shefi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\Shefi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\Shefi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The mi

C:\Users\Shefi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\Shefi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\Shefi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\Shefi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The mi

...

Evaluated 55 individuals
-- GENERATION 5 --


C:\Users\Shefi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\Shefi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\Shefi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\Shefi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The mi

C:\Users\Shefi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\Shefi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\Shefi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
C:\Users\Shefi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The mi

Evaluated 58 individuals
-- Only the fittest survives --
Best individual is [1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0], (0.99,)


C:\Users\Shefi\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


In [50]:
gen_columns =[1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1]

In [52]:
all_columns=df.columns
gen__columns=[]

for i in range(len(gen_columns)):
    if gen_columns[i] == 1:
        gen__columns.append(all_columns[i])

In [53]:
gen__columns

['Unnamed: 0',
 ' Source Port',
 ' Protocol',
 ' Flow Duration',
 ' Total Fwd Packets',
 ' Total Backward Packets',
 'Total Length of Fwd Packets',
 ' Fwd Packet Length Max',
 ' Fwd Packet Length Std',
 ' Bwd Packet Length Min',
 ' Bwd Packet Length Mean',
 ' Flow Packets/s',
 ' Flow IAT Mean',
 ' Flow IAT Max',
 ' Flow IAT Min',
 'Fwd IAT Total',
 ' Fwd IAT Mean',
 'Bwd IAT Total',
 ' Bwd IAT Std',
 ' Bwd IAT Max',
 ' Bwd IAT Min',
 ' Fwd URG Flags',
 ' Fwd Header Length',
 'Fwd Packets/s',
 ' Bwd Packets/s',
 ' Max Packet Length',
 ' Packet Length Mean',
 ' Packet Length Std',
 ' URG Flag Count',
 ' CWE Flag Count',
 ' Avg Fwd Segment Size',
 'Subflow Fwd Packets',
 ' Subflow Fwd Bytes',
 ' act_data_pkt_fwd',
 ' Active Min',
 'Idle Mean',
 ' Idle Max',
 ' Idle Min']

In [7]:
X = df.drop([' Label'], axis=1)
y = df[' Label']
print(X.head())

In [105]:
X.head()

,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Inbound
0,836,49970,17,1,2,0,2304.0,0.0,1152.0,1152.0,...,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,540,56876,17,1,2,0,2944.0,0.0,1472.0,1472.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,666,53544,17,1,2,0,458.0,0.0,229.0,229.0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,515,38921,17,1,2,0,458.0,0.0,229.0,229.0,...,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,913,20813,17,1,2,0,2944.0,0.0,1472.0,1472.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


## Random forest without Feature Selection

In [106]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [107]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
model = RandomForestClassifier(random_state=1)
model.fit(X_train, y_train)

/home/shashi/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [108]:
y_pred = model.predict(X_test)
accu = accuracy_score(y_test, y_pred)

In [109]:
y_pred

array(['MSSQL', 'UDP', 'MSSQL', ..., 'LDAP', 'MSSQL', 'Syn'], dtype=object)

In [110]:
accu = accuracy_score(y_test, y_pred)
print("Accuracy for the Random Forest Classifier {} %".format(round(accu*100,3)))

Accuracy for the Random Forest Classifier 87.767 %


In [112]:
classification_re = classification_report(y_test, y_pred)
print("\nClassification Scores: \n\n {}".format(classification_re))


Classification Scores: 

               precision    recall  f1-score   support

      BENIGN       0.99      1.00      0.99       330
        LDAP       1.00      1.00      1.00      7336
       MSSQL       0.99      0.99      0.99      7941
     NetBIOS       0.61      0.68      0.65      8716
     Portmap       0.59      0.51      0.54      7689
         Syn       1.00      1.00      1.00     13986
         UDP       0.99      0.99      0.99      9225
      UDPLag       0.60      0.60      0.60        20

    accuracy                           0.88     55243
   macro avg       0.84      0.85      0.84     55243
weighted avg       0.88      0.88      0.88     55243



In [56]:
df_new = pd.read_csv('final.csv')

C:\Users\Shefi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Random Forest with feature selection using Genetic Algorithm

In [57]:
y_new = df_new[' Label']

In [58]:
X_new = df_new.drop([' Label'], axis=1)

In [59]:
for i in X_new.columns:
    if i not in gen__columns:
        X_new.drop(i, axis=1, inplace=True)

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.33, random_state=1)

In [61]:
# fit the model
model = RandomForestClassifier(random_state=1)
model.fit(X_train, y_train)

C:\Users\Shefi\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [62]:
y_test

41033       MSSQL
134800        UDP
42814       MSSQL
120843        UDP
24915       MSSQL
           ...   
160942        Syn
86022     Portmap
5207         LDAP
46347       MSSQL
110391        Syn
Name:  Label, Length: 55243, dtype: object

In [63]:
y_pred = model.predict(X_test)

In [64]:
y_pred

array(['MSSQL', 'UDP', 'MSSQL', ..., 'LDAP', 'MSSQL', 'Syn'], dtype=object)

In [65]:
print(accuracy_score(y_test, y_pred))

0.9075176945495357


In [42]:
classification_report = classification_report(y_test, y_pred)
print("\nClassification Scores: \n\n {}".format(classification_report))


Classification Scores: 

               precision    recall  f1-score   support

      BENIGN       0.98      1.00      0.99       330
        LDAP       1.00      1.00      1.00      7336
       MSSQL       0.99      0.99      0.99      7941
     NetBIOS       0.71      0.73      0.72      8716
     Portmap       0.68      0.67      0.67      7689
         Syn       1.00      1.00      1.00     13986
         UDP       0.99      0.99      0.99      9225
      UDPLag       0.75      0.60      0.67        20

    accuracy                           0.91     55243
   macro avg       0.89      0.87      0.88     55243
weighted avg       0.91      0.91      0.91     55243

